In [156]:
#Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
from geopy.geocoders import Nominatim
from sklearn import preprocessing
import pickle
import math
import copy

#Path to datafile
datapath = "All_zip.csv"
WINDOW_SIZE = 50
LEARNING_RATE = 0.000025
N_EPOCHS = 200
HIDDEN_SIZE = 100
EPOCHS_EARLY = 20

In [157]:
#Data cleaning
class PriceData():

    def __init__(self, filename):
        self.df = self.generateDF(filename)

    def generateDF(self, filename):
        
        def lat(row):
            return geolocator.geocode({"postalcode":row["RegionName"]})[1][0]

        def long(row):
            return geolocator.geocode({"postalcode":row["RegionName"]})[1][1]

        
        orig_df = pd.read_csv(filename)
        filtered_df = orig_df.loc[(orig_df['State'] == "CA")]
        
        

        columns = filtered_df.columns
        

        cal_df = filtered_df \
            .dropna() \
            .drop(columns=columns[3:9]) \
            .drop(columns=columns[0:2]) \
            .rename(columns={'RegionName': 'ZipCode'}) \
            .reset_index(drop=True)
        
        
        cal_df.set_index("ZipCode", inplace = True)

#         for zip_code in cal_df.index:
#             print(zip_code)
#             cal_df["windows"] = cal_df.apply(lambda row: createBatches(row["prices"]), axis = 1)
        

        return cal_df

In [158]:
class LSTM(nn.Module):
    
    def __init__(self,input_size = 1, window_size = WINDOW_SIZE, hidden_size = HIDDEN_SIZE, out_size = 1, learning_rate = LEARNING_RATE, epochs = N_EPOCHS):

        super().__init__()
        
        #Model
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size,out_size)
        self.hidden = (torch.zeros(1,1,hidden_size),torch.zeros(1,1,hidden_size))
        self.window_size = None
        self.prices = None
        self.window_size = window_size
        
        #Data processing
        self.scaler = None
        
        #Training parameters
        self.criterion = nn.MSELoss()
        self.optimiser = torch.optim.Adam(self.parameters(), lr=learning_rate)
        
        
        #Hyperparameters
        self.epochs = epochs
        
        
        
    
    def _preprocessor(self, ySeries):
        y = ySeries.values

        
        
        window_size = self.window_size

        out = []
        self.prices = y[:window_size].reshape(window_size)
        for i in range(len(y) - window_size):
            scaler = preprocessing.MinMaxScaler()
            y_scaled = scaler.fit_transform(y[i:i+window_size+1].reshape(-1, 1))
            self.scaler = scaler
            window = torch.tensor(y_scaled[:window_size]).to(torch.float32)
            label = torch.tensor(y_scaled[window_size:window_size+1]).to(torch.float32)
            self.prices = np.append(self.prices, y[i+window_size:i+window_size+1][0])
            out.append((window, label))
        return out
    
    def fit(self, y, y_val):
        y_train = self._preprocessor(y)
        
        epochs_wo_imp = 0
        lowest_rmse = math.inf
        val_rmse = math.inf
        
        
        for epoch in range(self.epochs):
            for seq, y_hat in y_train:
                self.optimiser.zero_grad()
                self.hidden = (torch.zeros(1,1,self.hidden_size), torch.zeros(1,1,self.hidden_size))

                y_pred = self(seq)
                loss = self.criterion(y_hat[0], y_pred)
                loss.backward() 
                self.optimiser.step()
            
            val_rmse = self.score(y_val)[0]
            
            if (val_rmse < lowest_rmse):
                best_model = copy.deepcopy(self)
                best_model.prices = np.append(best_model.prices, y_val)
                lowest_rmse = val_rmse
                epochs_wo_imp = 0
            else:
                epochs_wo_imp += 1
                
            if (epochs_wo_imp == EPOCHS_EARLY):
                return best_model, lowest_rmse
            
            print(f'Epoch: {epoch+1:2} Test Loss: {loss.item():10.8f} Val RMSE: {val_rmse}')
            
        return (best_model, lowest_rmse)
                
            
    def predict(self, future_instances):
        
        self.eval()
        for i in range(future_instances):
            scaler = preprocessing.MinMaxScaler()
            seq = torch.FloatTensor(scaler.fit_transform(self.prices[-self.window_size:].reshape(-1, 1)))
            self.scaler = scaler
            with torch.no_grad():
                self.hidden = (torch.zeros(1,1,self.hidden_size),
                                torch.zeros(1,1,self.hidden_size))
                self.prices = np.append(self.prices, self.scaler.inverse_transform(np.array(self(seq).item()).reshape(-1, 1)))
        
        predictions = self.prices[-future_instances:]
        self.prices = self.prices[:-future_instances]
        return predictions
    
    def predict_inplace(self, future_instances):
        
        self.eval()
        for i in range(future_instances):
            scaler = preprocessing.MinMaxScaler()
            seq = torch.FloatTensor(scaler.fit_transform(self.prices[-self.window_size:].reshape(-1, 1)))
            self.scaler = scaler
            with torch.no_grad():
                self.hidden = (torch.zeros(1,1,self.hidden_size),
                                torch.zeros(1,1,self.hidden_size))
                self.prices = np.append(self.prices, self.scaler.inverse_transform(np.array(self(seq).item()).reshape(-1, 1)))
        
        predictions = self.prices[-future_instances:]
        return predictions
    
    def score(self, y):
        y_pred = self.predict(len(y))
        rmse = np.sqrt(np.mean(((np.array(y_pred)-np.array(y)))**2))
        return rmse, y_pred

        
        
    def forward(self,seq):
        lstm_out, self.hidden = self.lstm(seq.view(len(seq),1,-1), self.hidden)
        pred = self.linear(lstm_out.view(len(seq),-1))
        return pred[-1]

In [159]:
def split_dataset(y, valid_proportion, test_proportion):

   
    
    n_test = round(len(y) * test_proportion)
    n_valid = round(len(y) * valid_proportion)
    n_train = (len(y) - n_test) - n_valid
    
    y_train = y[:n_train]
    y_valid = y[n_train:n_train + n_valid]
    y_test = y[n_train + n_valid:]
    
    return (y_train, y_valid, y_test)


def save_regressor(trained_model, zipcode): 
    """ 
    Utility function to save the trained regressor model in part2_model.pickle.
    """
    # If you alter this, make sure it works in tandem with load_regressor
    with open(str(zipcode) + '_model.pickle', 'wb') as target:
        pickle.dump(trained_model, target)
    print(f"\nSaved model in {zipcode}_model.pickle\n")


def load_regressor(zipcode): 
    """ 
    Utility function to load the trained regressor model in part2_model.pickle.
    """
    # If you alter this, make sure it works in tandem with save_regressor
    with open(str(zipcode) + '_model.pickle', 'rb') as target:
        trained_model = pickle.load(target)
    print(f"\nLoaded model in {zipcode}_model.pickle\n")
    return trained_model

    

In [ ]:
def hyperparameterSearch( window_sizes = [10, 20, 40, 80, 100], hidden_sizes = [10, 20, 40, 60, 80, 100, 150], zipcode = 91331, df = cal_df):
    lowest_rmse = math.inf
    val_rmse = math.inf
    

    for window_size in window_sizes:
        WINDOW_SIZE = window_size
        for hidden_size in hidden_sizes:
            HIDDEN_SIZE = hidden_size
            
            print(f"Fitting for window_size of {WINDOW_SIZE} and hidden size of {HIDDEN_SIZE}")
            y = df.loc[zipcode]

            y_train, y_valid, y_test = split_dataset(y, 0.1, 0.1)

            model = LSTM(window_size = WINDOW_SIZE, hidden_size = HIDDEN_SIZE)
            model, val_rmse = model.fit(y_train, y_valid)
            print(f"Model trained for window_size of {model.window_size} and hidden size of {model.hidden_size}")


            if (val_rmse < lowest_rmse):
                best_model = copy.deepcopy(model)
                lowest_rmse = val_rmse
                
    save_regressor(best_model,zipcode)
    return best_model

def evaluate(zipcode):
    model = load_regressor(zipcode)
    print("Window size:", model.window_size, "Hidden size:", model.hidden_size)
    y = cal_df.loc[zipcode]
    y_train, y_valid, y_test = split_dataset(y, 0.1, 0.1)
    test_rmse, y_preds = model.score(y_test)
    print("RMSE:", test_rmse)


    plt.plot(np.append(model.prices, y_preds))
    plt.plot(np.append(y_train, np.append(y_valid, y_test)))
                

if __name__ == "__main__":
    cal_df = PriceData(datapath).df
    zip_codes = cal_df.index[:10]
    for zip_code in zip_codes:
        print(f"Training Model for ZipCode {zip_code}")
        hyperparameterSearch(zipcode = zip_code, df = cal_df)

        print(f"Evaluating Model for ZipCode {zip_code}")
        evaluate(zip_code)

Training Model for ZipCode 91331
Fitting for window_size of 10 and hidden size of 10
Epoch:  1 Test Loss: 1.51058602 Val RMSE: 2362288.205484327
Epoch:  2 Test Loss: 1.44213843 Val RMSE: 1320228.8402632435
Epoch:  3 Test Loss: 1.36966038 Val RMSE: 712916.8871378748
Epoch:  4 Test Loss: 1.29151356 Val RMSE: 376942.9920579836
Epoch:  5 Test Loss: 1.20627201 Val RMSE: 204457.57210925384
Epoch:  6 Test Loss: 1.11346793 Val RMSE: 121924.31169876434
Epoch:  7 Test Loss: 1.01379013 Val RMSE: 83344.67747244623
Epoch:  8 Test Loss: 0.90910482 Val RMSE: 66139.95306154968
Epoch:  9 Test Loss: 0.80238545 Val RMSE: 61764.87539011007
Epoch: 10 Test Loss: 0.69726127 Val RMSE: 58875.129665769826
Epoch: 11 Test Loss: 0.59728301 Val RMSE: 55902.139821731114
Epoch: 12 Test Loss: 0.50529706 Val RMSE: 52964.376489235416
Epoch: 13 Test Loss: 0.42317134 Val RMSE: 50155.88093018768
Epoch: 14 Test Loss: 0.35179853 Val RMSE: 47549.60265979362
Epoch: 15 Test Loss: 0.29123783 Val RMSE: 45239.3116582156
Epoch: 16 

Epoch: 13 Test Loss: 0.05302816 Val RMSE: 35341.402308702825
Epoch: 14 Test Loss: 0.04889445 Val RMSE: 35171.11329490745
Epoch: 15 Test Loss: 0.04586950 Val RMSE: 35048.78315418236
Epoch: 16 Test Loss: 0.04348618 Val RMSE: 34954.908045976736
Epoch: 17 Test Loss: 0.04146402 Val RMSE: 34877.782844445464
Epoch: 18 Test Loss: 0.03963109 Val RMSE: 34811.08413521075
Epoch: 19 Test Loss: 0.03787971 Val RMSE: 34749.83041298077
Epoch: 20 Test Loss: 0.03614033 Val RMSE: 34691.57696135539
Epoch: 21 Test Loss: 0.03436737 Val RMSE: 34635.25432462831
Epoch: 22 Test Loss: 0.03253040 Val RMSE: 34580.300639250476
Epoch: 23 Test Loss: 0.03061021 Val RMSE: 34527.425111048855
Epoch: 24 Test Loss: 0.02859600 Val RMSE: 34477.606842105335
Epoch: 25 Test Loss: 0.02648491 Val RMSE: 34434.22155531706
Epoch: 26 Test Loss: 0.02428084 Val RMSE: 34397.926012810676
Epoch: 27 Test Loss: 0.02199494 Val RMSE: 34378.39978058168
Epoch: 28 Test Loss: 0.01964542 Val RMSE: 34383.619061478516
Epoch: 29 Test Loss: 0.01725889 

Epoch: 66 Test Loss: 0.00249016 Val RMSE: 30478.863130513208
Epoch: 67 Test Loss: 0.00232321 Val RMSE: 30420.29856380392
Epoch: 68 Test Loss: 0.00216654 Val RMSE: 30383.18648385103
Epoch: 69 Test Loss: 0.00202161 Val RMSE: 30334.24888555612
Epoch: 70 Test Loss: 0.00188903 Val RMSE: 30270.546251467473
Epoch: 71 Test Loss: 0.00176852 Val RMSE: 30227.2131460326
Epoch: 72 Test Loss: 0.00165958 Val RMSE: 30245.985877687337
Epoch: 73 Test Loss: 0.00156138 Val RMSE: 30278.663908477807
Epoch: 74 Test Loss: 0.00147303 Val RMSE: 30313.3015075947
Epoch: 75 Test Loss: 0.00139363 Val RMSE: 30264.779559157323
Epoch: 76 Test Loss: 0.00132230 Val RMSE: 30165.759529271647
Epoch: 77 Test Loss: 0.00125823 Val RMSE: 30081.683099414415
Epoch: 78 Test Loss: 0.00120065 Val RMSE: 30010.73500529284
Epoch: 79 Test Loss: 0.00114887 Val RMSE: 29951.385599034988
Epoch: 80 Test Loss: 0.00110231 Val RMSE: 29901.847502250883
Epoch: 81 Test Loss: 0.00106035 Val RMSE: 29861.138165490454
Epoch: 82 Test Loss: 0.00102255 

Epoch: 29 Test Loss: 0.00520094 Val RMSE: 31992.878339546067
Epoch: 30 Test Loss: 0.00468481 Val RMSE: 31993.548904763255
Epoch: 31 Test Loss: 0.00417687 Val RMSE: 31935.020415100913
Epoch: 32 Test Loss: 0.00369300 Val RMSE: 31877.063442609026
Epoch: 33 Test Loss: 0.00324383 Val RMSE: 31788.64695702169
Epoch: 34 Test Loss: 0.00283600 Val RMSE: 31710.311006381773
Epoch: 35 Test Loss: 0.00247285 Val RMSE: 31653.01905561962
Epoch: 36 Test Loss: 0.00215505 Val RMSE: 31629.711076016127
Epoch: 37 Test Loss: 0.00188112 Val RMSE: 31623.415070461775
Epoch: 38 Test Loss: 0.00164801 Val RMSE: 31572.47288715313
Epoch: 39 Test Loss: 0.00145180 Val RMSE: 31533.639993765566
Epoch: 40 Test Loss: 0.00128809 Val RMSE: 31503.418170945086
Epoch: 41 Test Loss: 0.00115247 Val RMSE: 31480.445719613774
Epoch: 42 Test Loss: 0.00104073 Val RMSE: 31462.74423039708
Epoch: 43 Test Loss: 0.00094908 Val RMSE: 31433.19914318957
Epoch: 44 Test Loss: 0.00087421 Val RMSE: 31410.964678894434
Epoch: 45 Test Loss: 0.000813

Epoch: 21 Test Loss: 0.05086565 Val RMSE: 37881.31344392029
Epoch: 22 Test Loss: 0.04798919 Val RMSE: 37579.46745122627
Epoch: 23 Test Loss: 0.04578548 Val RMSE: 37342.39712207906
Epoch: 24 Test Loss: 0.04406230 Val RMSE: 37152.49939631917
Epoch: 25 Test Loss: 0.04268366 Val RMSE: 36996.939429374645
Epoch: 26 Test Loss: 0.04155359 Val RMSE: 36866.45405621474
Epoch: 27 Test Loss: 0.04060379 Val RMSE: 36754.96464617776
Epoch: 28 Test Loss: 0.03978537 Val RMSE: 36656.69925921538
Epoch: 29 Test Loss: 0.03906291 Val RMSE: 36568.03912927384
Epoch: 30 Test Loss: 0.03841097 Val RMSE: 36486.35495587535
Epoch: 31 Test Loss: 0.03781074 Val RMSE: 36409.679993600446
Epoch: 32 Test Loss: 0.03724862 Val RMSE: 36336.56158725577
Epoch: 33 Test Loss: 0.03671443 Val RMSE: 36265.91083917094
Epoch: 34 Test Loss: 0.03620072 Val RMSE: 36196.908192461306
Epoch: 35 Test Loss: 0.03570199 Val RMSE: 36128.95435669574
Epoch: 36 Test Loss: 0.03521401 Val RMSE: 36061.584520684366
Epoch: 37 Test Loss: 0.03473365 Val 

...

Epoch:  8 Test Loss: 0.05763557 Val RMSE: 67896.55543406871
Epoch:  9 Test Loss: 0.05188502 Val RMSE: 66006.70903017728
Epoch: 10 Test Loss: 0.04690650 Val RMSE: 64211.51597859719
Epoch: 11 Test Loss: 0.04248295 Val RMSE: 62459.045272895455
Epoch: 12 Test Loss: 0.03851082 Val RMSE: 60732.52822457125
Epoch: 13 Test Loss: 0.03492451 Val RMSE: 59024.890824293674
Epoch: 14 Test Loss: 0.03167508 Val RMSE: 57332.14981666453
Epoch: 15 Test Loss: 0.02872267 Val RMSE: 55651.04279199874
Epoch: 16 Test Loss: 0.02603375 Val RMSE: 53978.52620075993
Epoch: 17 Test Loss: 0.02358036 Val RMSE: 52312.09320680364
Epoch: 18 Test Loss: 0.02133975 Val RMSE: 50650.5585669061
Epoch: 19 Test Loss: 0.01929395 Val RMSE: 48994.18973577108
Epoch: 20 Test Loss: 0.01742887 Val RMSE: 47345.24272260011
Epoch: 21 Test Loss: 0.01573347 Val RMSE: 45707.91223144356
Epoch: 22 Test Loss: 0.01419770 Val RMSE: 44086.76137069655
Epoch: 23 Test Loss: 0.01281185 Val RMSE: 42486.25863757665
Epoch: 24 Test Loss: 0.01156480 Val RMS

Epoch: 14 Test Loss: 0.01854885 Val RMSE: 49213.58688199715
Epoch: 15 Test Loss: 0.01504345 Val RMSE: 45332.486925244724
Epoch: 16 Test Loss: 0.01201441 Val RMSE: 41298.95551083883
Epoch: 17 Test Loss: 0.00942980 Val RMSE: 37171.92973877951
Epoch: 18 Test Loss: 0.00726145 Val RMSE: 33042.97678389978
Epoch: 19 Test Loss: 0.00547964 Val RMSE: 29033.71590918188
Epoch: 20 Test Loss: 0.00404931 Val RMSE: 25122.806017722774
Epoch: 21 Test Loss: 0.00292937 Val RMSE: 20755.010109279272
Epoch: 22 Test Loss: 0.00207455 Val RMSE: 16418.51944137194
Epoch: 23 Test Loss: 0.00143806 Val RMSE: 12183.020894999408
Epoch: 24 Test Loss: 0.00097470 Val RMSE: 8049.99386883253
Epoch: 25 Test Loss: 0.00064381 Val RMSE: 4201.186650863037
Epoch: 26 Test Loss: 0.00041149 Val RMSE: 2925.659034844098
Epoch: 27 Test Loss: 0.00025151 Val RMSE: 6537.942516104814
Epoch: 28 Test Loss: 0.00014454 Val RMSE: 11312.72713687439
Epoch: 29 Test Loss: 0.00007626 Val RMSE: 16367.457428128766
Epoch: 30 Test Loss: 0.00003537 Val 

Epoch: 61 Test Loss: 0.11766805 Val RMSE: 108943.79787096272
Epoch: 62 Test Loss: 0.11269159 Val RMSE: 107666.99382002266
Epoch: 63 Test Loss: 0.10783725 Val RMSE: 106348.53960883935
Epoch: 64 Test Loss: 0.10311413 Val RMSE: 104991.22287201072
Epoch: 65 Test Loss: 0.09852995 Val RMSE: 103598.16962608547
Epoch: 66 Test Loss: 0.09409103 Val RMSE: 102173.0890008641
Epoch: 67 Test Loss: 0.08980227 Val RMSE: 100719.98438393597
Epoch: 68 Test Loss: 0.08566739 Val RMSE: 99243.32640876173
Epoch: 69 Test Loss: 0.08168885 Val RMSE: 97747.66359190636
Epoch: 70 Test Loss: 0.07786836 Val RMSE: 96238.22304598411
Epoch: 71 Test Loss: 0.07420641 Val RMSE: 94720.17345741899
Epoch: 72 Test Loss: 0.07070275 Val RMSE: 93198.79862379604
Epoch: 73 Test Loss: 0.06735629 Val RMSE: 91679.44311853196
Epoch: 74 Test Loss: 0.06416509 Val RMSE: 90167.34962375516
Epoch: 75 Test Loss: 0.06112676 Val RMSE: 88667.50962291879
Epoch: 76 Test Loss: 0.05823823 Val RMSE: 87184.77493244565
Epoch: 77 Test Loss: 0.05549548 Va

Epoch: 196 Test Loss: 0.00372594 Val RMSE: 44703.381957322315
Epoch: 197 Test Loss: 0.00365064 Val RMSE: 44636.98938715856
Epoch: 198 Test Loss: 0.00357599 Val RMSE: 44565.95644412503
Epoch: 199 Test Loss: 0.00350200 Val RMSE: 44490.20062470454
Epoch: 200 Test Loss: 0.00342859 Val RMSE: 44408.34885682616
Model trained for window_size of 100 and hidden size of 10
Fitting for window_size of 100 and hidden size of 20
Epoch:  1 Test Loss: 1.27642393 Val RMSE: 4209148.378440883
Epoch:  2 Test Loss: 1.21205103 Val RMSE: 2516076.6675971798
Epoch:  3 Test Loss: 1.15032375 Val RMSE: 1529743.5452381505
Epoch:  4 Test Loss: 1.08916378 Val RMSE: 936985.3286919626
Epoch:  5 Test Loss: 1.02752578 Val RMSE: 579447.2448979482
Epoch:  6 Test Loss: 0.96484101 Val RMSE: 371917.8227933355
Epoch:  7 Test Loss: 0.90089220 Val RMSE: 257542.52533641743
Epoch:  8 Test Loss: 0.83571315 Val RMSE: 201928.56124592212
Epoch:  9 Test Loss: 0.76958966 Val RMSE: 193668.94346624432
Epoch: 10 Test Loss: 0.70310587 Val R

Epoch: 130 Test Loss: 0.00013727 Val RMSE: 5081.048283204679
Epoch: 131 Test Loss: 0.00014268 Val RMSE: 5162.626137714789
Epoch: 132 Test Loss: 0.00014811 Val RMSE: 5250.878844664875
Epoch: 133 Test Loss: 0.00015354 Val RMSE: 5344.75450459575
Epoch: 134 Test Loss: 0.00015895 Val RMSE: 5442.729040920131
Epoch: 135 Test Loss: 0.00016431 Val RMSE: 5542.715317689213
Epoch: 136 Test Loss: 0.00016957 Val RMSE: 5642.536810787925
Epoch: 137 Test Loss: 0.00017474 Val RMSE: 5740.846334749342
Epoch: 138 Test Loss: 0.00017978 Val RMSE: 5835.727582348241
Epoch: 139 Test Loss: 0.00018468 Val RMSE: 5926.713843939971
Model trained for window_size of 100 and hidden size of 20
Fitting for window_size of 100 and hidden size of 40
Epoch:  1 Test Loss: 0.73064280 Val RMSE: 179874.56796056958
Epoch:  2 Test Loss: 0.68200237 Val RMSE: 175223.9579190083
Epoch:  3 Test Loss: 0.63334918 Val RMSE: 170429.06428221898
Epoch:  4 Test Loss: 0.58307511 Val RMSE: 165318.042320237
Epoch:  5 Test Loss: 0.53081667 Val RM